In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
source_object = ""

In [0]:

cdc_col = "modified_date"
backdated_refresh = ""
target_schema = "gold"
target_object = "dim_flights"
source_schema = "silver"
source_object = "silver_flights"
key_cols_list = eval('["flight_id"]')
surrogate_key = "dim_flights_key"

if not backdated_refresh:
    if spark.catalog.tableExists(f"workspace.{target_schema}.{target_object}"):
        last_load = spark.sql(f"select max({cdc_col}) from workspace.{target_schema}.{target_object}").collect()[0][0]

    else:
        last_load = "1900-01-01 00:00:00"
else:
    last_load = backdated_refresh

last_load

In [0]:
if spark.catalog.tableExists(f"workspace.{target_schema}.{target_object}"):

    key_cols_string_incremental = ", ".join(key_cols_list)
    df_target = spark.sql(f"SELECT {key_cols_string_incremental}, {surrogate_key}, create_date, update_date FROM workspace.{target_schema}.{target_object}")

else:

    key_cols_string_init = ", ".join([f"'' AS {i}" for i in key_cols_list])

    df_target = spark.sql(
        f"SELECT {key_cols_string_init}, 0 as {surrogate_key}, CAST('1900-01-01 00:00:00' as timestamp) as create_date, CAST('1900-01-01 00:00:00' as timestamp) as update_date FROM (SELECT 1) tmp WHERE 1=0"
    )



In [0]:
df_src = spark.sql(f"SELECT * FROM {source_schema}.{source_object} WHERE {cdc_col} > '{last_load}'")

In [0]:
df_src.display()

In [0]:
join_condition = " AND ".join(f"src.{i} = trg.{i}" for i in key_cols_list)


df_src.createOrReplaceTempView("src")
df_target.createOrReplaceTempView("trg")

df_join = spark.sql(f"""
          SELECT src.*,
            trg.{surrogate_key},
            trg.create_date,
            trg.update_date
          FROM src
          LEFT JOIN trg
          ON {join_condition}
          
          """)

In [0]:
df_join.display()

In [0]:
# old records
df_old = df_join.filter(col(f'{surrogate_key}').isNotNull())
df_old_enriched = df_old.withColumn('update_date', current_timestamp())

# new records
df_new = df_join.filter(col(f'{surrogate_key}').isNull())

In [0]:
if spark.catalog.tableExists(f"workspace.{target_schema}.{target_object}"):
    max_surrogate_key = spark.sql(
        f"SELECT max({surrogate_key}) FROM workspace.{target_schema}.{target_object}"
                                  ).collect()[0][0]
    df_new_enriched = df_new.withColumn(f"{surrogate_key}", lit(max_surrogate_key) + 1 + monotonically_increasing_id())\
        .withColumn('create_date', current_timestamp())\
        .withColumn('update_date', current_timestamp())

else:
    df_new_enriched = df_new.withColumn(f"{surrogate_key}", 1 + monotonically_increasing_id())\
        .withColumn('create_date', current_timestamp())\
        .withColumn('update_date', current_timestamp())


In [0]:
max_surrogate_key

In [0]:
df_union = df_old_enriched.unionByName(df_new_enriched)

df_union.display()

In [0]:
# upsert
# modified_date is used as the condition for upsert - source of truth for records

from delta.tables import DeltaTable

if spark.catalog.tableExists(f"workspace.{target_schema}.{target_object}"):

    dlt_object = DeltaTable.forName(spark, f"workspace.{target_schema}.{target_object}")

    dlt_object.alias("trg").merge(df_union.alias("src"),
        f"trg.{surrogate_key} = src.{surrogate_key}"
    ).whenMatchedUpdateAll(condition=f"src.{cdc_col} >= trg.{cdc_col}")\
        .whenNotMatchedInsertAll()\
            .execute()

else:

    df_union.write\
        .format("delta")\
            .mode("append")\
                .saveAsTable(f"workspace.{target_schema}.{target_object}")


In [0]:
%sql
SELECT * FROM `workspace`.`gold`.`dim_flights`

In [0]:
# Records are upserted correctly 
#  